In [ ]:
!pip install phenograph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow.keras.losses import MSE, KLD
import phenograph

from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
# no
tf.random.set_seed(123)

In [ ]:
def myscatter(Y, class_idxs, legend=False, ran=True, seed=229):
    if ran:
        np.random.seed(seed)
    Y = np.array(Y)
    fig, ax = plt.subplots(figsize=(5,4), dpi=300)
    classes = list(np.unique(class_idxs))
    markers = 'osD' * len(classes)
    colors = plt.cm.rainbow(np.linspace(0, 1, len(classes)))
    if ran:
        np.random.shuffle(colors)

    for i, cls in enumerate(classes):
        mark = markers[i]
        ax.plot(Y[class_idxs == cls, 0], Y[class_idxs == cls, 1], marker=mark,
                linestyle='', ms=4, label=str(cls), alpha=1, color=colors[i],
                markeredgecolor='black', markeredgewidth=0.15)
    if legend:
        ax.legend(bbox_to_anchor=(1.03, 1), loc=2, borderaxespad=0, fontsize=10, markerscale=2, frameon=False,
                  ncol=2, handletextpad=0.1, columnspacing=0.5)

    plt.xticks([])
    plt.yticks([])

    return ax

def dotsne(X, dim=2, ran=23):
    tsne = TSNE(n_components=dim, random_state=ran)
    Y_tsne = tsne.fit_transform(X)
    return Y_tsne

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/idents1_raw_1.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/data1_raw_1_SCT.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/idents3_raw_1.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/data3_raw_1_SCT.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/ileum_cell_info.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/ileum_raw_SCT.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/colon_cell_info.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/colon_raw_SCT.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/pbmc_idents.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/pbmc_raw_SCT.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/bm4/bm4_idents.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/bm4/bm4_raw_SCT.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/mousebrain/mousebrain_idents.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/mousebrain/mousebrain_raw_SCT.csv", index_col=0, sep="\t"))

In [ ]:
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/IPMN_SCT.csv", index_col=0, sep="\t"))

In [ ]:
import os
os.chdir("/content/drive/My Drive/compbio/test_data/")

In [ ]:
count = np.load("pbmc_pysct.npy")
idents = np.load("pbmc_idents.npy")

In [ ]:
count = np.load("ileum_norm.npy")
idents = np.load("ileum_idents.npy")

In [ ]:
count = np.load("IPMN_pysct.npy")

In [ ]:
count = np.transpose(count)
idents = idents.flatten()
[n_sample, n_gene] = count.shape
x_train = count.astype('float32')
idents = idents.astype('str')

In [ ]:
count = np.transpose(count)
[n_sample, n_gene] = count.shape
x_train = count.astype('float32')
idents = np.zeros(n_sample)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
def clustering(h, n_cluster, k=15, f="kmeans"):
    if f == "kmeans":
        labels = KMeans(n_clusters=n_cluster, random_state=0).fit(h).labels_
    elif f == "spectral":
        labels = SpectralClustering(n_clusters=n_cluster, affinity="precomputed", assign_labels="discretize",
                                    random_state=0).fit_predict(adj)
    
    return labels

In [ ]:
import keras
from keras.layers import Layer
class DenseTranspose(Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias", initializer="zeros",
                                      shape=[self.dense.input_shape[-1]])
        super().build(batch_input_shape)
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
        return self.activation(z + self.biases)

In [ ]:
from keras import backend as K

def _nan2zero(x):
    return tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)

def _nan2inf(x):
    return tf.where(tf.math.is_nan(x), tf.zeros_like(x)+np.inf, x)

def cal_zinb(pi=x_train, disp=x_train, ridge_lambda=0.0, x_pred0=x_train, x_true0=x_train):
    x_true = tf.cast(x_true0, tf.float32)
    x_pred = tf.cast(x_pred0, tf.float32)
    
    eps = 1e-10
    theta = tf.minimum(disp, 1e6)
    t1 = tf.math.lgamma(theta+eps) + tf.math.lgamma(x_true+1.0) - tf.math.lgamma(x_true+theta+eps)
    t2 = (theta+x_true) * tf.math.log(1.0 + (x_pred/(theta+eps))) + (x_true * (tf.math.log(theta+eps) - tf.math.log(x_pred+eps)))
    final = t1 + t2
    final = _nan2inf(final)

    nb_case = final - tf.math.log(1.0-pi+eps)
    zero_nb = tf.math.pow(theta/(theta+x_pred+eps), theta)
    zero_case = -tf.math.log(pi + ((1.0-pi)*zero_nb)+eps)
    result = tf.where(tf.less(x_true, 1e-8), zero_case, nb_case)
    ridge = ridge_lambda*tf.square(pi)
    result += ridge

    result = tf.reduce_mean(result)

    result = _nan2inf(result)

    return result

In [ ]:
from tensorflow.keras import activations, constraints, initializers, regularizers
from tensorflow.keras.layers import Layer, InputSpec
import tensorflow.keras.backend as K

class ClusteringLayer(Layer):
    def __init__(self, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=None, initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def target_distribution(q):
    q = q.numpy()
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [ ]:
# no
clustering_layer = ClusteringLayer(name='clustering', input_shape=(25,))
clustering = keras.models.Sequential([clustering_layer])

In [ ]:
def cal_cluster(hidden, clusters, alpha):
    clusters = tf.convert_to_tensor(clusters, dtype=tf.float32)
    q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(hidden, axis=1) - clusters), axis=2) / alpha))
    q **= (alpha + 1.0) / 2.0
    q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
    return q

def cal_dist(hidden, clusters):
    clusters = tf.convert_to_tensor(clusters, dtype=tf.float32)
    dist1 = K.sum(K.square(K.expand_dims(hidden, axis=1) - clusters), axis=2)
    temp_dist1 = dist1 - tf.reshape(tf.reduce_min(dist1, axis=1), [-1, 1])
    temp_dist1 = K.transpose(K.transpose(temp_dist1) / K.max(temp_dist1, axis=1))
    q = K.exp(-temp_dist1 * 10)
    #q = K.exp(-temp_dist1)
    q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
    q = K.pow(q, 2)
    q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
    dist2 = dist1 * q
    return dist1, dist2

def cal_latent(hidden, alpha):
    sum_y = K.sum(K.square(hidden), axis=1)
    num = -2.0 * tf.matmul(hidden, tf.transpose(hidden)) + tf.reshape(sum_y, [-1, 1]) + sum_y
    num = num / alpha
    num = tf.pow(1.0 + num, -(alpha + 1.0) / 2.0)
    zerodiag_num = num - tf.linalg.diag(tf.linalg.diag_part(num))
    latent_p = K.transpose(K.transpose(zerodiag_num) / K.sum(zerodiag_num, axis=1))
    return num, latent_p

def target_dis(latent_p):
    latent_q = tf.transpose(tf.transpose(tf.pow(latent_p, 2)) / tf.reduce_sum(latent_p, axis = 1))
    return tf.transpose(tf.transpose(latent_q) / tf.reduce_sum(latent_q, axis = 1))

def computeCentroids(data, labels):
    n_clusters = len(np.unique(labels))
    data_1 = np.array(data)
    return np.array([data_1[labels == i].mean(0) for i in range(n_clusters)])

def get_centers(Y):
    #from sklearn.cluster import SpectralClustering
    l, _,  _ = phenograph.cluster(Y)
    #l = KMeans(n_clusters=10, random_state=0).fit(Y).labels_
    centers = computeCentroids(Y, l)
    return centers, l

def get_centers_kmeans(Y, k):
    #from sklearn.cluster import SpectralClustering
    #l, _,  _ = phenograph.cluster(Y)
    l = KMeans(n_clusters=k, random_state=0).fit(Y).labels_
    centers = computeCentroids(Y, l)
    return centers, l

In [ ]:
from scipy.optimize import linear_sum_assignment

def make_matrix(a, b, n):
    mat = np.zeros([n, n])
    x = np.zeros(n)
    y = np.zeros(n)
    for i in range(n):
        x[i] = a[a==i].size
        y[i] = b[b==i].size
    #print(x)
    #print(y)
    for i in range(n):
        for j in range(n):
            x0 = x[i]
            y0 = y[j]
            mat[i,j] = 1 - min(x0,y0) / max(x0, y0)
    return mat

def update_labels(l, mat):
    x,y = linear_sum_assignment(mat)
    n = len(l)
    dic = dict(zip(x,y))
    #print(dic)
    for i in range(n):
        l[i] = dic[l[i]]
    return l

In [ ]:
class AE(tf.keras.Model):

    def __init__(self, encoder, decoder, zinb_model):
        super(AE, self).__init__()
        self.ec = encoder
        self.dc = decoder
        #self.clustering = clustering
        self.nb = zinb_model

In [ ]:
from tensorflow.keras.losses import MSE, KLD
import math
from keras.layers import Lambda


class SAE():

    def __init__(self, X):
        super(SAE, self).__init__()
        encoded1 = Dense(512, activation = "relu", kernel_constraint = keras.constraints.NonNeg(), input_shape=(n_gene,), use_bias=True,
                 kernel_initializer='glorot_uniform',bias_initializer='zeros')
        encoded2 = Dense(256, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(512,), use_bias=True,
                 kernel_initializer='glorot_uniform',bias_initializer='zeros')
        encoded3 = Dense(128, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(256,), use_bias=True,
                 kernel_initializer='glorot_uniform',bias_initializer='zeros')
        encoded4 = Dense(25, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(128,), use_bias=True,
                 kernel_initializer='glorot_uniform',bias_initializer='zeros')

        encoder = keras.models.Sequential([encoded1, encoded2, encoded3, encoded4])
        encoder1 = keras.models.Sequential([encoded1])
        encoder2 = keras.models.Sequential([encoded2])
        encoder3 = keras.models.Sequential([encoded3])
        encoder4 = keras.models.Sequential([encoded4])

        decoded1 = DenseTranspose(encoded4, activation = "relu")
        decoded2 = DenseTranspose(encoded3, activation = "relu")
        decoded3 = DenseTranspose(encoded2, activation = "relu")
        decoded4 = DenseTranspose(encoded1, activation = "relu")

        decoder = keras.models.Sequential([decoded1, decoded2, decoded3, decoded4])
        decoder1 = keras.models.Sequential([decoded1])
        decoder2 = keras.models.Sequential([decoded2])
        decoder3 = keras.models.Sequential([decoded3])
        decoder4 = keras.models.Sequential([decoded4])

        autoencoder1 = keras.models.Sequential([encoded1, decoded4])
        autoencoder2 = keras.models.Sequential([encoded2, decoded3])
        autoencoder3 = keras.models.Sequential([encoded3, decoded2])
        autoencoder4 = keras.models.Sequential([encoded4, decoded1])

        self.X = X
        self.ec1 = encoder1
        self.ec2 = encoder2
        self.ec3 = encoder3
        self.ec4 = encoder4
        self.dc1 = decoder1
        self.dc2 = decoder2
        self.dc3 = decoder3
        self.dc4 = decoder4
        self.autoencoder1 = autoencoder1
        self.autoencoder2 = autoencoder2
        self.autoencoder3 = autoencoder3
        self.autoencoder4 = autoencoder4
        self.ec = encoder
        self.dc = decoder
        self.autoencoder = keras.models.Sequential([encoder, decoder])
        self.clustering = clustering



        MeanAct = lambda x: tf.clip_by_value(K.exp(x), 1e-5, 1e6)
        DispAct = lambda x: tf.clip_by_value(tf.nn.softplus(x), 1e-4, 1e4)

        y = Input(shape=(512,))
        pi = Dense(n_gene, activation='sigmoid', name='pi')(y)
        disp = Dense(n_gene, activation=DispAct, name='dispersion')(y)
        mean = Dense(n_gene, activation=MeanAct, name='mean')(y)

        zinb_model = keras.models.Model(inputs=[y], outputs=[pi, disp, mean])

        self.AE = AE(encoder, decoder, zinb_model)
        self.nb = zinb_model
        
        
    def train1(self, max_epoch=100, lr=0.002):
        optimizer = tf.keras.optimizers.Adam(lr)
        for epoch in range(0, max_epoch):
            with tf.GradientTape(persistent=True) as tape:
                h = self.ec1(self.X)
                z = self.dc4(h)
                loss = tf.reduce_mean(MSE(count, z))
            vars = self.autoencoder1.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 10 == 0:
                print(loss)
        print("Finish!")
        return h
        
    def train2(self, h, max_epoch=120, lr=0.005):
        optimizer = tf.keras.optimizers.Adam(lr)
        for epoch in range(0, max_epoch):
            with tf.GradientTape(persistent=True) as tape:
                h1 = self.ec2(h)
                z = self.dc3(h1)
                loss = tf.reduce_mean(MSE(h, z))
            vars = self.autoencoder2.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 50 == 0:
                print(loss)
        print("Finish!")
        return h1
    
    def train3(self, h, max_epoch=350, lr=0.01):
        optimizer = tf.keras.optimizers.Adam(lr)
        for epoch in range(0, max_epoch):
            with tf.GradientTape(persistent=True) as tape:
                h1 = self.ec3(h)
                z = self.dc2(h1)
                loss = tf.reduce_mean(MSE(h, z))

            vars = self.autoencoder3.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 50 == 0:
                print(loss)
        print("Finish!")
        return h1
        
    def train4(self, h, max_epoch=350, lr=0.01):
        optimizer = tf.keras.optimizers.Adam(lr)
        for epoch in range(0, max_epoch):
            with tf.GradientTape(persistent=True) as tape:
                h1 = self.ec4(h)
                z = self.dc1(h1)
                loss = tf.reduce_mean(MSE(h, z))

            vars = self.autoencoder4.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 50 == 0:
                print(loss)
        print("Finish!")
        return h1

    def train(self, max_epoch=100, lr=0.001):
        weight0 = self.AE.get_weights()
        optimizer = tf.keras.optimizers.Adam(lr)
        for epoch in range(1, max_epoch+1):
            with tf.GradientTape(persistent=True) as tape:
                
                h = self.ec(self.X)
                z = self.dc(h)

                a = K.eval(tf.math.is_nan(h))
                b = K.eval(tf.math.is_inf(h))
                if a.any() or b.any():
                    #print(self.AE.weights)
                    self.AE.set_weights(weight0)
                    #print(self.AE.weights)
                    break;

                z1 = self.dc1(h)
                z1 = self.dc2(z1)
                z1 = self.dc3(z1)
                pi, disp, mean = self.nb(z1)
                zinb_loss = cal_zinb(pi, disp, 0, z, x_train)

                #loss = tf.reduce_mean(MSE(count, z)) + 0.30 * zinb_loss
                loss = tf.reduce_mean(MSE(count, z))
                weight0 = self.AE.get_weights()
            
            vars = self.AE.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 10 == 0:
                print(epoch)
                print(loss)
                print(zinb_loss)
        print("Finish!")
    
    def clustering_train(self):
        self.autoencoder1.trainable = False
        self.autoencoder2.trainable = False
        self.autoencoder3.trainable = False

        k = 8
        h = self.ec(self.X)
        h1 = self.dc1(h)
        centers, labels = get_centers(np.array(h))
        labels1 = KMeans(n_clusters=k, random_state=0).fit(h1).labels_
        labels2 = KMeans(n_clusters=k, random_state=0).fit(h).labels_
        mat = make_matrix(labels1, labels2, k)
        labels1 = update_labels(labels1, mat)
        centers1 = computeCentroids(h1, labels1)
        centers2 = computeCentroids(h, labels2)

        print(h.shape)
        print(h1.shape)

        q = cal_cluster(h, centers, 1.0)
        p = target_distribution(q)
        optimizer = tf.keras.optimizers.Adam()
        for epoch in range(1, 51):
            if epoch % 10 == 0:
                centers, labels = get_centers(np.array(h))
                labels1 = KMeans(n_clusters=k, random_state=0).fit(h1).labels_
                labels2 = KMeans(n_clusters=k, random_state=0).fit(h).labels_
                mat = make_matrix(labels1, labels2, k)
                labels1 = update_labels(labels1, mat)
                centers1 = computeCentroids(h1, labels1)
                centers2 = computeCentroids(h, labels2)

                print(labels1[0:10])
                print(labels2[0:10])

                q = cal_cluster(h, centers, 1.0)
                p = target_distribution(q)
                q1 = cal_cluster(h1, centers1, 1.0)
                q2 = cal_cluster(h, centers2, 1.0)

            with tf.GradientTape(persistent=True) as tape:
                h = self.ec(self.X)
                z = self.dc(h)
                loss = tf.reduce_mean(MSE(count, z))
                
                # clustering loss
                q_out = cal_cluster(h, centers, 1.0)
                cluster_loss = tf.reduce_mean(KLD(q_out, p))
                loss = loss + 2.5 * cluster_loss

                # k-means center loss
                latent_dist1, latent_dist2 = cal_dist(h, centers)
                kmeans_loss = tf.reduce_mean(tf.reduce_sum(latent_dist2, axis=1))
                loss = loss + 0.1 * kmeans_loss

                # cell-cell KLD loss
                num, latent_p = cal_latent(h, 1)
                latent_q = target_dis(latent_p)
                latent_p = latent_p + tf.linalg.diag(tf.linalg.diag_part(num))
                latent_q = latent_q + tf.linalg.diag(tf.linalg.diag_part(num))
                kl_loss = tf.reduce_mean(KLD(latent_p, latent_q))
                loss = loss + 2.5 * kl_loss

                # Consistency loss
                h1 = self.dc1(h)
                q1 = cal_cluster(h1, centers1, 1.0)
                q2 = cal_cluster(h, centers2, 1.0)
                if q2.shape == q1.shape:
                    cluster_loss_1 = tf.reduce_mean(KLD(q2, q1))
                else:
                    cluster_loss_1 = 0
                loss = loss + 2.5 * cluster_loss_1
            vars = self.autoencoder.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 10 == 0:
                print(loss)
                print(cluster_loss)
                print(kmeans_loss)
                print(kl_loss)
                print(cluster_loss_1)
        print("Finish!")
        

In [ ]:
autoencoder = SAE(x_train)

In [ ]:
h = autoencoder.train1()

In [ ]:
h = autoencoder.train2(h)

In [ ]:
h = autoencoder.train3(h)

In [ ]:
h = autoencoder.train4(h)

In [ ]:
autoencoder.train()

In [ ]:
autoencoder.clustering_train()

In [ ]:
encoded_data = autoencoder.ec(x_train)
ed = np.array(encoded_data)
ed = dotsne(ed)

In [ ]:
myscatter(ed, idents, legend=True)

In [ ]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, homogeneity_score, completeness_score
def measure(true, pred):
    NMI = round(normalized_mutual_info_score(true, pred), 2)
    RAND = round(adjusted_rand_score(true, pred), 2)
    HOMO = round(homogeneity_score(true, pred), 2)
    COMP = round(completeness_score(true, pred), 2)
    return [NMI, RAND, HOMO, COMP]

In [ ]:
labels, _,  _ = phenograph.cluster(np.array(encoded_data))
measure(idents, labels)

In [ ]:
myscatter(ed,labels, legend=True)

In [ ]:
labels = KMeans(n_clusters=7, random_state=0).fit(ed).labels_
measure(idents, labels)

In [ ]:
myscatter(ed,labels, legend=True)

In [ ]:
ed = dotsne(count)
myscatter(ed, idents, legend=True)

In [ ]:
pca = decomposition.PCA(n_components = 2)
pca.fit(count)
count_pca = pca.transform(count)

In [ ]:
myscatter(count_pca, idents, legend=True)

In [ ]:
save = pd.DataFrame(labels)
save.to_csv('/content/drive/My Drive/compbio/test_data/bm4_predict.csv',index=False,header=True)

In [ ]:
save = pd.DataFrame(np.array(encoded_data))
save.to_csv('/content/drive/My Drive/compbio/test_data/bm4_encoded.csv',index=False,header=True)

In [ ]:
save = pd.DataFrame(ed)
save.to_csv('/content/drive/My Drive/compbio/test_data/bm4_2d.csv',index=False,header=True)

In [ ]:
!pip install hungarian-algorithm

In [ ]:
a = np.array([0,1,2,1,0,3,3,3,3,4,4])
b = np.array([2,1,0,1,2,3,3,3,4,4,4])

In [ ]:
import torch
from torch import nn, optim
from torchtext import data, datasets
 
if torch.cuda.is_available:
  print('PyTorch can use GPUs!')
else:
  print('PyTorch cannot use GPUs.') 

In [ ]:
y_pred = x_train
y_true = x_train
eps = 1e-10
scale_factor=1.0
theta = 1e6
t1 = tf.math.lgamma(theta+eps) + tf.math.lgamma(y_true+1.0) - tf.math.lgamma(y_true+theta+eps)
t2 = (theta+y_true) * tf.math.log(1.0 + (y_pred/(theta+eps))) + (y_true * (tf.math.log(theta+eps) - tf.math.log(y_pred+eps)))
final = t1 + t2
final = _nan2inf(final)

In [ ]:
def _nan2inf(x):
    return tf.where(tf.math.is_nan(x), tf.zeros_like(x)+np.inf, x)

In [ ]:
final

In [ ]:
pi = 0.1
nb_case = final - tf.math.log(1.0-pi+eps)
zero_nb = tf.math.pow(theta/(theta+y_pred+eps), theta)
zero_case = -tf.math.log(pi + ((1.0-pi)*zero_nb)+eps)
result = tf.where(tf.less(y_true, 1e-8), zero_case, nb_case)
ridge = 0*tf.square(pi)
result += ridge
result = tf.reduce_mean(result)
result = _nan2inf(result)

In [ ]:
nb_case

In [ ]:
!pip install scanpy

In [ ]:
import anndata as ad
import scanpy as sc

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/idents3_raw_1.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/data3_raw_1.csv", index_col=0, sep=","))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/idents1_raw_1.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/data1_raw_1.csv", index_col=0, sep=","))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/pbmc_idents.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/pbmc_raw.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/colon_cell_info.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/colon_raw.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/ileum_cell_info.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/ileum_raw.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/bm4_idents.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/bm4_raw.csv", index_col=0, sep="\t"))

In [ ]:
count = np.array(pd.read_csv("/content/drive/My Drive/compbio/test_data/IPMN_raw.csv", index_col=0, sep="\t"))

In [ ]:
count = np.transpose(count)

In [ ]:
adata=sc.AnnData(count)

In [ ]:
adata

In [ ]:
sc.pp.normalize_per_cell(adata)

In [ ]:
count = adata.X

In [ ]:
count.shape

In [ ]:
from sklearn.preprocessing import normalize, scale

In [ ]:
count_norm = scale(count)
count = np.transpose(count_norm)

In [ ]:
count_norm.shape

In [ ]:
np.var(count, axis=0)

In [ ]:
np.var(normalize(count), axis=1)

In [ ]:
np.var(normalize(count), axis=0)

In [ ]:
np.var(normalize(np.transpose(count)), axis=1)

In [ ]:
np.var(normalize(np.transpose(count)), axis=0)

In [ ]:
count = np.transpose(count)

In [ ]:
np.sum(count, axis=0)

In [ ]:
!pip install scanpy louvain
!pip install git+https://github.com/saketkc/pysctransform.git

In [ ]:
from pysctransform import vst, get_hvg_residuals, SCTransform
import anndata as ad
import scanpy as sc
from scipy.sparse import *

In [ ]:
# colon, ileum, bm4 no transpose
# pbmc, sim transpose

In [ ]:
adata=sc.AnnData(count)
adata.X = csr_matrix(adata.X)
residuals = SCTransform(adata, var_features_n=3000)

In [ ]:
count = np.array(residuals)

In [ ]:
import os
os.chdir("/content/drive/My Drive/compbio/test_data/")

In [ ]:
np.save('ileum_norm.npy',count)

In [ ]:
np.save('IPMN_pysct.npy',count)

In [ ]:
np.save('ileum_idents.npy',idents)

In [ ]:
res = residuals.columns.values
res = res.astype('int32')
res.sort()
count = np.transpose(count)
count0 = count[:,res]
adata = ad.AnnData(np.array(residuals), dtype='float32')
adata.layers["counts"] = count0
adata.raw = adata

In [ ]:
adata = ad.AnnData(count, dtype='int32')
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata

In [ ]:
adata.X